# Homework - RAG Intro

In [ ]:
# pip install minsearch

## Import Libraries

In [ ]:
import minsearch
import json
import json
from elasticsearch import Elasticsearch
from tqdm import tqdm
from mistralai import Mistral
import os
import requests 

## Download  Data File

In [ ]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Testing the Minsearch package

In [ ]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [ ]:
query = "Can I join the course if it has already started?"

filter_dict = {"course": "data-engineering-zoomcamp"}
boost_dict = {"question": 3}

results = index.search(query, filter_dict, boost_dict, num_results=5)

for result in results:
    print(json.dumps(result, indent=2))

## Question 1

In [ ]:
es_client = Elasticsearch('http://localhost:9200') 

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

## Question 2

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, body = doc)

100%|██████████| 948/948 [00:05<00:00, 167.71it/s]


## Question 3

In [ ]:
query = "How do execute a command on a Kubernetes pod?"
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", 
                                   "text", 
                                   #"section"
                                  ],
                        "type": "best_fields"
                    }
                },
#                 "filter": {
#                     "term": {
#                         "course": "data-engineering-zoomcamp"
#                     }
#                 }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        res_tuple = []
        res_tuple.append(hit['_source'])
        res_tuple.append(hit['_score'])
        result_docs.append(res_tuple)
        
    
    return result_docs

In [ ]:
es_client

In [ ]:
res_query = elastic_search(query)

In [ ]:
res_query[0]

## Question 4

In [ ]:
query = "How do copy a file to a Docker container?"
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", 
                                   "text", 
                                   #"section"
                                  ],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        res_tuple = []
        res_tuple.append(hit['_source'])
        res_tuple.append(hit['_score'])
        result_docs.append(res_tuple)
        
    
    return result_docs

In [ ]:
res_query = elastic_search(query)
res_query[2]

## Question 5

In [ ]:

api_key = os.getenv("mistralai_key")
model = "mistral-large-latest"

client = Mistral(api_key=api_key)

messages = [
    {"role": "user", "content": "What is the best French cheese?"}
]

response = client.chat.complete(
    model=model,
    messages=messages
)

print(response.choices[0].message.content)

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", 
                                   "text", 
                                   #"section"
                                  ],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    context_template = """
Q: {question}
A: {text}\n\n
"""
    
    for doc in search_results:
        context = context  + context_template.format(question = doc['question'], text = doc['text']) 
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)
print(prompt)

In [ ]:
def llm(prompt):
    response = client.chat.complete(  # Use .complete() method
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer
query = "How do I execute a command in a running docker container?"
answer_q = rag(query)

In [ ]:
answer_q

In [ ]:
len(answer_q)

## Question 6

In [ ]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(prompt))